In [1]:
# Crawling the top 5 market capitalization of game stocks

import pandas as pd
import numpy as np

import requests
import datetime

import time

from bs4 import BeautifulSoup
from selenium import webdriver
from html_table_parser import parser_functions as parser

In [2]:
# Chrom driver coonnected
driver = webdriver.Chrome('../driver/chromedriver.exe')

In [3]:
# 업종별 시세 url
driver.get('https://finance.naver.com/sise/sise_group_detail.nhn?type=upjong&no=235')

In [4]:
# 업종별 시세
# 추출하고 싶은 항목 체크 수정 (6개 사항만 조회할 수 있음)

# 기본으로 선택되어있는 항목 체크 해제
vol_btn_xpath='//*[@id="option1"]'
buy_btn_xpath='//*[@id="option2"]'
amount_btn_xpath='//*[@id="option3"]'
sell_btn_xpath='//*[@id="option8"]'
pre_vol_btn_xpath='//*[@id="option9"]'

# 추출하고 싶은 항목 체크
market_cap_btn_xpath='//*[@id="option4"]'
per_btn_xpath='//*[@id="option6"]'

# 적용하기 xpath
apply_btn_xpath='//*[@id="contentarea"]/div[3]/form/div/div/div/a[1]/img'

In [5]:
# 기본 항목 체크 해제
driver.find_element_by_xpath(vol_btn_xpath).click()
driver.find_element_by_xpath(buy_btn_xpath).click()
driver.find_element_by_xpath(amount_btn_xpath).click()
driver.find_element_by_xpath(sell_btn_xpath).click()
driver.find_element_by_xpath(pre_vol_btn_xpath).click()

# 추출하고 싶은 항목 체크
driver.find_element_by_xpath(market_cap_btn_xpath).click()
driver.find_element_by_xpath(per_btn_xpath).click()

# 적용하기 클릭
driver.find_element_by_xpath(apply_btn_xpath).click()

In [6]:
# 테이블 찾기
company_table_xpath='//*[@id="contentarea"]/div[4]/table'

# element에 접근 => tr, td에 접근한 후 for문으로 각 행에 접근해 갈 예정
company_table = driver.find_element_by_xpath(company_table_xpath)

In [7]:
# beutifulSoup
html = driver.page_source # 현재 렌더링 된 페이지의 Elements를 모두 가져옴
# page_source : 브라우저에 보이는 그대로의 HTML, 크롬 개발자 도구의 Element 탭 내용과 동일

soup = BeautifulSoup(html, 'html.parser')

table = soup.find('table',attrs={'summary':'업종별 시세 리스트'})

In [8]:
# url
url_base = 'https://finance.naver.com/'
each_company = table.find_all('div', attrs={'class':'name_area'})

# 각각의 회사로 들어가는 리스트 (url_each)
url_each = [url_base+company_name.find('a')['href'] for company_name in each_company]

In [9]:
# a태그에서 회사이름(name) 추출
name = [company_name.find('a').get_text() for company_name in each_company]
# get_text는 태그안의 내용 추출

In [10]:
trs = table.find_all('tr') # 테이블 별 각각의 항목(tr)을 모두 가져옴

# nan이 아닌것만 담기 위해 1이상인 것만
market_cap = [tr.find_all('td', class_='number')[3].text.strip() for tr in trs if len(tr.find_all('td', class_='number'))>=1]
per = [tr.find_all('td', class_='number')[4].text.strip() for tr in trs if len(tr.find_all('td', class_='number'))>=1]

In [11]:
code = [] # 종목코드
reg_day = [] # 상장일
class_name=[] # 코스닥인지 코스피인지

In [12]:
for url in url_each:
    driver.get(url)
    
    # 코스닥/코스피가 이미지로 구분됨
    kospi_xpath = '//*[@id="middle"]/div[1]/div[1]/div/img'
    # class에 코스닥인지 코스피인지 명시되어 있음
    kospi_or_kospaq= driver.find_element_by_xpath(kospi_xpath).get_attribute('class')
    
    class_name.append(kospi_or_kospaq)
    stock_code=driver.find_element_by_xpath('//*[@id="middle"]/div[1]/div[1]/div/span[1]').text
    code.append(stock_code)
    
    # 상장일 가져오기
    # stock_analysis_btn_xpath='//*[@id="content"]/ul/li[6]/a/span' # 종목분석
    # company_summary_btn_xpath='//*[@id="header-menu"]/div[1]/dl/dt[2]/a' # 기업 개요

    # 종목분석 클릭
    # driver.find_element_by_xpath(stock_analysis_btn_xpath).click()
    # 왜 xpath로 안되는 걸까... 개빡침....
    # driver.find_element_by_xpath(company_summary_btn_xpath).click()

    company_summary_link = 'https://navercomp.wisereport.co.kr/v2/company/c1020001.aspx?cmp_cd={code}&cn='.format(code=stock_code)
    driver.get(company_summary_link)

    driver.implicitly_wait(3)

    # 상장일이 포함된 리스트 출력
    reg_date = driver.find_element_by_xpath('//*[@id="cTB201"]/tbody/tr[3]/td[1]').text
    reg_day.append(reg_date[-11:-1])

In [13]:
print(code)
print(reg_day)
print(class_name)

['225570', '060240', '042420', '950180', '192080', '063080', '205500', '251270', '052790', '293490', '056000', '052770', '181710', '095660', '067000', '030350', '950190', '201490', '123420', '036570', '047080', '078340', '023770', '069080', '263750', '217270', '058630', '112040', '041140', '215000', '299910', '206400', '194480', '101730', '208640']
['2015/09/25', '2002/01/04', '2000/06/27', '2019/05/07', '2019/03/12', '2009/07/30', '2014/11/12', '2017/05/12', '2001/08/14', '2020/09/10', '2003/07/11', '2002/08/22', '2013/08/29', '2007/07/02', '2008/05/30', '1997/11/10', '2020/08/18', '2016/10/10', '2010/11/10', '2003/05/22', '2002/01/10', '2007/07/06', '1994/11/07', '2003/05/23', '2017/09/14', '2015/06/18', '2008/12/19', '2009/12/18', '2000/07/25', '2015/04/03', '2018/12/03', '2014/12/10', '2014/10/06', '2009/06/03', '2014/12/29']
['kosdaq', 'kosdaq', 'kosdaq', 'kosdaq', 'kospi', 'kosdaq', 'kosdaq', 'kospi', 'kosdaq', 'kosdaq', 'kosdaq', 'kosdaq', 'kospi', 'kosdaq', 'kosdaq', 'kosdaq', 

- 종목 코드 : code
- 코스피인지 코스닥인지 : class
(내용물은 'kospi' 또는 'kosdaq' 으로 기입)
- 종목 이름 : name
- 시가총액 : market_cap
- 상장일 : reg_day
- PER : per (소수점 두 번째 자리까지)

In [14]:
# 데이터 프레임으로 저장
cols = ['code', 'name', 'class','reg_day', 'market_cap', 'per']

data_df = pd.DataFrame(
    {
          'code':code
        , 'name':name
        , 'class':class_name
        , 'reg_day':reg_day
        , 'market_cap':market_cap
        , 'per':per
    }
)

In [15]:
driver.close()

In [16]:
data_df

,code,name,class,reg_day,market_cap,per
0,225570,넷게임즈,kosdaq,2015/09/25,"4,198",16.08
1,060240,룽투코리아,kosdaq,2002/01/04,"1,688",58.76
2,042420,네오위즈홀딩스,kosdaq,2000/06/27,"3,383",6.54
3,950180,SNK,kosdaq,2019/05/07,"5,866",55.81
4,192080,더블유게임즈,kospi,2019/03/12,"12,587",11.18
5,063080,게임빌,kosdaq,2009/07/30,"3,185",15.75
6,205500,액션스퀘어,kosdaq,2014/11/12,992,-48.98
7,251270,넷마블,kospi,2017/05/12,"109,574",34.96
8,052790,액토즈소프트,kosdaq,2001/08/14,"1,040",6.67
9,293490,카카오게임즈,kosdaq,2020/09/10,"38,677",37.64


In [17]:
# 데이터 , 제거
# 람다를 사용해서 제거
data_df['market_cap'] = data_df['market_cap'].apply(lambda x:x.replace(",",""))

In [18]:
data_df

,code,name,class,reg_day,market_cap,per
0,225570,넷게임즈,kosdaq,2015/09/25,4198,16.08
1,060240,룽투코리아,kosdaq,2002/01/04,1688,58.76
2,042420,네오위즈홀딩스,kosdaq,2000/06/27,3383,6.54
3,950180,SNK,kosdaq,2019/05/07,5866,55.81
4,192080,더블유게임즈,kospi,2019/03/12,12587,11.18
5,063080,게임빌,kosdaq,2009/07/30,3185,15.75
6,205500,액션스퀘어,kosdaq,2014/11/12,992,-48.98
7,251270,넷마블,kospi,2017/05/12,109574,34.96
8,052790,액토즈소프트,kosdaq,2001/08/14,1040,6.67
9,293490,카카오게임즈,kosdaq,2020/09/10,38677,37.64


In [19]:
# 날짜 데이터 데이터 타입 변환
data_df['reg_day'] = pd.to_datetime(data_df['reg_day'])

In [20]:
# 시가총액(market_cap)
data_df['market_cap'] = data_df['market_cap'].astype('int')
data_df['market_cap']

0       4198
1       1688
2       3383
3       5866
4      12587
5       3185
6        992
7     109574
8       1040
9      38677
10       564
11       345
12     14028
13      5255
14      4347
15       385
16      3316
17      2104
18      2120
19    193415
20      1576
21     20985
22       851
23     14089
24     42486
25      8381
26      1737
27     10601
28      4705
29      5673
30      2191
31      1401
32     11254
33      1158
34      4857
Name: market_cap, dtype: int32

In [35]:
# data_df.to_csv('game_entertainment_full.csv')

In [26]:
# 조건 1. 설립한지 10년 이상
day_diff = datetime.datetime.now() - data_df['reg_day']
day = day_diff.dt.days # 현재로부터 상장일을 뺀 일수만 출력
day_condition = day >= (365*10)
day_condition

0     False
1      True
2      True
3     False
4     False
5      True
6     False
7     False
8      True
9     False
10     True
11     True
12    False
13     True
14     True
15     True
16    False
17    False
18     True
19     True
20     True
21     True
22     True
23     True
24    False
25    False
26     True
27     True
28     True
29    False
30    False
31    False
32    False
33     True
34    False
Name: reg_day, dtype: bool

In [27]:
# 코스딕?
kosdaq_kospi_condition = (data_df['class']== kospi_or_kospaq)
kosdaq_kospi_condition

0      True
1      True
2      True
3      True
4     False
5      True
6      True
7     False
8      True
9      True
10     True
11     True
12    False
13     True
14     True
15     True
16     True
17     True
18     True
19    False
20     True
21     True
22     True
23     True
24     True
25     True
26     True
27     True
28     True
29     True
30     True
31     True
32     True
33     True
34     True
Name: class, dtype: bool

In [29]:
# 시가총액 10위안,기업 가져오는 코드
collected_data = data_df[kosdaq_kospi_condition & day_condition]

In [30]:
top_market = collected_data.sort_values(by='market_cap', ascending=False)

In [31]:
top_market

,code,name,class,reg_day,market_cap,per
21,078340,컴투스,kosdaq,2007-07-06,20985,26.11
23,069080,웹젠,kosdaq,2003-05-23,14089,16.35
27,112040,위메이드,kosdaq,2009-12-18,10601,-144.72
13,095660,네오위즈,kosdaq,2007-07-02,5255,8.68
28,041140,넥슨지티,kosdaq,2000-07-25,4705,118.75
14,067000,조이시티,kosdaq,2008-05-30,4347,34.93
2,042420,네오위즈홀딩스,kosdaq,2000-06-27,3383,6.54
5,063080,게임빌,kosdaq,2009-07-30,3185,15.75
18,123420,선데이토즈,kosdaq,2010-11-10,2120,14.48
26,058630,엠게임,kosdaq,2008-12-19,1737,25.55


In [34]:
# 상위 5개 기업
top_5_market = top_market[:5]

In [36]:
top_5_market.to_csv('game_entertainment.csv',encoding='utf-8')